In [97]:
import numpy as np
import pandas as pd 
import seaborn as sns
import pickle 
import regex as re
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [3]:
data_set = pickle.load(open( "full_data.pickle", "rb" ))

In [4]:
data_set #this is already sorted well -last column is y 

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSGR0000270726.4,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,index,id,drug_name,vital_status
0,14.125848,0.093522,30.585785,1.278731,1.062113,0.764395,1.199922,17.554991,6.304686,5.136213,...,0.0,0.0,0.0,0.0,0.0,0.0,455fd7d4-5ff4-423f-83ce-4630aef89bb7,455fd7d4-5ff4-423f-83ce-4630aef89bb7,sorafenib,dead
1,22.776941,2.057021,24.838891,2.461498,0.461543,2.207669,8.707784,29.256859,4.376688,6.046493,...,0.0,0.0,0.0,0.0,0.0,0.0,455fd7d4-5ff4-423f-83ce-4630aef89bb7,455fd7d4-5ff4-423f-83ce-4630aef89bb7,sorafenib,dead
2,19.023014,6.957998,15.983518,1.650498,0.330992,1.450858,5.505354,26.203638,4.645046,5.568023,...,0.0,0.0,0.0,0.0,0.0,0.0,822cf6c1-dd65-4814-94b1-0c335208ad9b,822cf6c1-dd65-4814-94b1-0c335208ad9b,oncophage vaccine,dead
3,12.314532,0.130207,26.086970,1.598978,0.365375,6.342112,3.582689,33.370253,4.451278,4.956177,...,0.0,0.0,0.0,0.0,0.0,0.0,822cf6c1-dd65-4814-94b1-0c335208ad9b,822cf6c1-dd65-4814-94b1-0c335208ad9b,oncophage vaccine,dead
4,11.800582,0.152335,39.217148,4.760523,0.863156,1.044859,13.206980,31.103094,8.694065,8.340499,...,0.0,0.0,0.0,0.0,0.0,0.0,9fc40b03-0b56-4b1f-93ed-dbc455ae3fd9,9fc40b03-0b56-4b1f-93ed-dbc455ae3fd9,interferon,dead
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,5.330827,0.805405,27.660611,1.546114,1.721013,4.393303,10.847711,44.014800,2.779097,6.636879,...,0.0,0.0,0.0,0.0,0.0,0.0,00eaaf62-c43e-4555-9134-1e5593386fdb,00eaaf62-c43e-4555-9134-1e5593386fdb,ifosfamide,dead
492,5.330827,0.805405,27.660611,1.546114,1.721013,4.393303,10.847711,44.014800,2.779097,6.636879,...,0.0,0.0,0.0,0.0,0.0,0.0,00eaaf62-c43e-4555-9134-1e5593386fdb,00eaaf62-c43e-4555-9134-1e5593386fdb,gemcitabine,dead
493,5.330827,0.805405,27.660611,1.546114,1.721013,4.393303,10.847711,44.014800,2.779097,6.636879,...,0.0,0.0,0.0,0.0,0.0,0.0,00eaaf62-c43e-4555-9134-1e5593386fdb,00eaaf62-c43e-4555-9134-1e5593386fdb,docetaxel,dead
494,5.330827,0.805405,27.660611,1.546114,1.721013,4.393303,10.847711,44.014800,2.779097,6.636879,...,0.0,0.0,0.0,0.0,0.0,0.0,00eaaf62-c43e-4555-9134-1e5593386fdb,00eaaf62-c43e-4555-9134-1e5593386fdb,temozolomide,dead


In [5]:
list_of_drugs = data_set['drug_name'].value_counts() #can probably split into gemcitabine and sunitinib and then other - maybe sorafenib


In [6]:
data_set[data_set['drug_name']=='gemictiabine'] #this is a misspelling
data_set.loc[277,'drug_name']='gemcitabine'

In [7]:
list_of_drugs[40:80] #can i group any of these?  #could try to seperate out the immunostuff
#gemzar is another name for gemcitabine - done 
#need to remove [not available]
#drop anything in () or after a space
#cisplatinum is a misspelling of cisplatin
#standardize andything with interferon
#leave tyrosine kinase inhibitor alone
#doxil is doxorubicin in a liposome

tivozanib                    2
tamoxifen                    2
iressa                       2
rad001                       2
alimta                       2
il-2 (high dose)             2
torisel                      2
et-743                       2
gefitinib                    2
ptk-zk                       1
inf                          1
bay-439006                   1
perifosine                   1
axitinib                     1
tyrosine kinase inhibitor    1
megace                       1
femara                       1
cisplatinum                  1
bortezomib                   1
ap-23573 (ariad)             1
etoposide                    1
ezn-2968                     1
gemictiabine                 1
enoticumab                   1
nab-rapamycin                1
letrozole                    1
capecitabin                  1
doxirubicin                  1
vorinostat                   1
liposomal doxorubicin        1
cytoxan                      1
psma dna vaccine             1
dtic, da

In [8]:
data_set['drug_name'] = data_set['drug_name'].str.replace(r"\s*\(.*\)\s*","")

In [9]:
gemcitabine_alias = ['Abine', 'Accogem', 'Acytabin', 'Antoril', 'axigem', 'Bendacitabin', 'Biogem', 'Boligem', 'Celzar', 'Citegin', 'Cytigem', 'Cytogem', 'Daplax', 'DBL', 'Demozar', 'Dercin', 'Emcitab', 'Enekamub', 'Eriogem', 'Fotinex', 'Gebina', 'Gemalata', 'Gembin', 'Gembine', 'Gembio', 'Gemcel', 'Gemcetin', 'Gemcibine', 'Gemcikal', 'Gemcipen', 'Gemcired', 'Gemcirena', 'Gemcit', 'Gemcitabin', 'Gemcitabina', 'Gemcitabine', 'Gemcitabinum', 'Gemcitan', 'Gemedac', 'Gemflor', 'Gemful', 'Gemita', 'Gemko', 'Gemliquid', 'Gemmis', 'Gemnil', 'Gempower', 'Gemsol', 'Gemstad', 'Gemstada', 'Gemtabine', 'Gemtavis', 'Gemtaz', 'Gemtero', 'Gemtra', 'Gemtro', 'Gemvic', 'Gemxit', 'Gemzar', 'Gentabim', 'Genuten', 'Genvir', 'Geroam', 'Gestredos', 'Getanosan', 'Getmisi', 'Gezt', 'Gitrabin', 'Gramagen', 'Haxanit', 'Jemta', 'Kalbezar', 'Medigem', 'Meditabine', 'Nabigem', 'Nallian', 'Oncogem', 'Oncoril', 'Pamigeno', 'Ribozar', 'Santabin', 'Sitagem', 'Symtabin', 'Yu Jie', 'Ze Fei', 'Zefei']
gemcitabine_alias= [name.lower() for name in gemcitabine_alias]
#gemcitabine_alias

In [10]:
data_set['drug_name']=['gemcitabine' if name in gemcitabine_alias else name for name in data_set.drug_name]

In [11]:
remove_index = data_set[(data_set.drug_name == '[not available]')|(data_set.drug_name == '[unknown]')].index #index values 89 347 444
remove_index
data_set.drop(remove_index, inplace=True)

In [78]:
data_set['drug_name']=['sorafenib' if 'nexavar' in name else name for name in data_set.drug_name]
data_set['drug_name']=['sorafenib' if 'bay-439006' in name else name for name in data_set.drug_name]
data_set['drug_name']=['sorafenib' if 'nexavaar' in name else name for name in data_set.drug_name]
data_set['drug_name']=['doxorubicin' if 'adriamycin' in name else name for name in data_set.drug_name]
data_set['drug_name']=['doxorubicin' if 'dox' in name else name for name in data_set.drug_name]
data_set['drug_name']=['doxorubicin' if 'rubex' in name else name for name in data_set.drug_name]
data_set['drug_name']=['sorafenib' if name=='naxavar' else name for name in data_set.drug_name]
data_set['drug_name']=['pazonib' if name=='votrient' else name for name in data_set.drug_name]
data_set['drug_name']=['sunitinib' if name=='sutent' else name for name in data_set.drug_name]
data_set['drug_name']=['docetaxel' if name=='taxotere' else name for name in data_set.drug_name]
data_set['drug_name']=['temsirolimus' if name=='torisel' else name for name in data_set.drug_name]
data_set['drug_name']=['avastin' if name=='bevacizumab' else name for name in data_set.drug_name]
data_set['drug_name']=['interferon alpha'if 'interferon' in name else name for name in data_set.drug_name]
data_set['drug_name']=['capecitabine' if 'xeloda' in name else name for name in data_set.drug_name]

data_set['drug_name']=['capecitabine' if 'capec' in name else name for name in data_set.drug_name]

data_set['drug_name']=['il-2' if 'il-2' in name else name for name in data_set.drug_name]
data_set['drug_name']=['interferon alpha' if '-a' in name else name for name in data_set.drug_name]
data_set['drug_name']=['cisplatin' if 'cisplat' in name else name for name in data_set.drug_name]
data_set['drug_name']=['everolimus' if 'afinitor' in name else name for name in data_set.drug_name]
data_set['drug_name']=['everolimus' if 'rad001' in name else name for name in data_set.drug_name]
data_set['drug_name']=['onocophage vaccine' if 'oncophage' in name else name for name in data_set.drug_name]
data_set['drug_name']=['trabectedin' if 'et-743' in name else name for name in data_set.drug_name]
data_set['drug_name']=['gefitinib' if 'iressa' in name else name for name in data_set.drug_name]
data_set['drug_name']=['dacarbazine' if 'dtic' in name else name for name in data_set.drug_name]
data_set['drug_name']=['axitinib' if 'axit' in name else name for name in data_set.drug_name]
data_set['drug_name']=['letrozole' if 'femara' in name else name for name in data_set.drug_name]
data_set['drug_name']=['pazopanib' if 'pazo' in name else name for name in data_set.drug_name]
#this can be cahnged to a dictionary later - its like this cause I was researching one drug at a time


In [13]:
#data_set(regex = '\w*\W*2', axis = 0)
data_set.drug_name = data_set.drug_name.str.replace(pat ="i\w*\W*2\w*", repl = 'il-2', regex=True)

In [88]:
data_set[data_set.drug_name=='evofosfamide']

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSGR0000270726.4,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,index,id,drug_name,vital_status


In [109]:
data_set.drug_name.value_counts().index[0] #set dead to 0, alive to 1 

'gemcitabine'

In [18]:
data_set['vital_status']=[0 if status=='dead' else 1 for status in data_set.vital_status]

In [19]:
data_set.vital_status.value_counts()

0    263
1    229
Name: vital_status, dtype: int64

In [107]:
data_set.groupby(['drug_name']).vital_status.value_counts()

drug_name       vital_status
5-fluorouracil  0                1
alimta          0                1
                1                1
ap-23573        0                1
avastin         0               22
                                ..
valproic acid   1                1
vincristine     0                1
vorinostat      0                1
zd6474          1                1
zometa          1                2
Name: vital_status, Length: 80, dtype: int64

In [89]:
# top 3 drugs
# gemcitabine            62
# sunitinib              60
# doxorubicin            46

In [90]:
gemcitabine_data = data_set[data_set.drug_name=='gemcitabine']

In [91]:
sunitinib_data = data_set[data_set.drug_name=='sunitinib']

In [92]:
doxorubicin_data = data_set[data_set.drug_name=='doxorubicin']

In [94]:
gemcitabine_data.vital_status.value_counts() #relatively even split!

1    38
0    24
Name: vital_status, dtype: int64

In [98]:
gemcitabine_data

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSGR0000270726.4,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,index,id,drug_name,vital_status
19,21.023010,4.698284,22.983041,2.520023,0.469418,1.552196,13.924939,28.691723,5.467230,7.892208,...,0.0,0.0,0.0,0.0,0.0,0.0,3fa6c93e-e7fe-402c-9526-c81411aa0920,3fa6c93e-e7fe-402c-9526-c81411aa0920,gemcitabine,0
20,19.843534,0.241418,10.711546,1.465249,0.497112,0.930400,28.637315,27.371243,1.936201,3.725314,...,0.0,0.0,0.0,0.0,0.0,0.0,3fa6c93e-e7fe-402c-9526-c81411aa0920,3fa6c93e-e7fe-402c-9526-c81411aa0920,gemcitabine,0
38,8.908702,0.114636,20.933515,1.901142,0.717593,3.416073,8.744393,28.441160,3.210180,8.010185,...,0.0,0.0,0.0,0.0,0.0,0.0,b7994839-77a5-4725-aa58-a9e8c79dbfc2,b7994839-77a5-4725-aa58-a9e8c79dbfc2,gemcitabine,0
49,11.133349,0.000000,20.704402,2.044744,0.560496,2.262629,0.892103,47.752124,9.209692,9.787083,...,0.0,0.0,0.0,0.0,0.0,0.0,2939c03a-6f3f-4f7b-b246-34361baadeb9,2939c03a-6f3f-4f7b-b246-34361baadeb9,gemcitabine,0
50,18.431117,0.177102,23.960874,1.849370,0.928399,0.932157,3.645760,28.621363,3.833655,6.840575,...,0.0,0.0,0.0,0.0,0.0,0.0,2939c03a-6f3f-4f7b-b246-34361baadeb9,2939c03a-6f3f-4f7b-b246-34361baadeb9,gemcitabine,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,7.864384,0.328180,61.969671,1.743994,1.115160,2.224356,17.646799,38.213663,2.887524,6.884513,...,0.0,0.0,0.0,0.0,0.0,0.0,719fa3f6-0194-423d-8a96-3709e8180565,719fa3f6-0194-423d-8a96-3709e8180565,gemcitabine,1
480,4.803510,0.953010,9.981320,0.635729,0.215869,0.354423,1.941930,14.554487,0.884119,1.699907,...,0.0,0.0,0.0,0.0,0.0,0.0,12bfd446-2ee5-4664-a492-c023d134f60f,12bfd446-2ee5-4664-a492-c023d134f60f,gemcitabine,1
488,1.295512,8.346813,27.546295,1.839392,1.886007,2.115162,0.950923,25.674871,1.891005,7.008985,...,0.0,0.0,0.0,0.0,0.0,0.0,4dc28966-d2c4-4661-b71f-d7f85e536cb6,4dc28966-d2c4-4661-b71f-d7f85e536cb6,gemcitabine,1
492,5.330827,0.805405,27.660611,1.546114,1.721013,4.393303,10.847711,44.014800,2.779097,6.636879,...,0.0,0.0,0.0,0.0,0.0,0.0,00eaaf62-c43e-4555-9134-1e5593386fdb,00eaaf62-c43e-4555-9134-1e5593386fdb,gemcitabine,0


In [95]:
sunitinib_data.vital_status.value_counts() #also even

0    39
1    21
Name: vital_status, dtype: int64

In [96]:
doxorubicin_data.vital_status.value_counts()

1    31
0    15
Name: vital_status, dtype: int64

In [101]:
gem_x = gemcitabine_data.iloc[:,0:-4]
gem_y = gemcitabine_data.iloc[:,-1]

In [102]:
gem_y

19     0
20     0
38     0
49     0
50     0
      ..
474    1
480    1
488    1
492    0
495    1
Name: vital_status, Length: 62, dtype: int64

In [104]:
gem_X_train, gem_X_test, gem_y_train, gem_y_test = train_test_split(gem_x, gem_y, test_size=0.20, random_state=42)

In [110]:
drug_list = data_set.drug_name.value_counts().index[0:3]

In [112]:
def split_dataframe_by_drug(drug_list, data_set):
    list_of_drug_dfs = []
    for drug in drug_list:
        list_of_drug_dfs.append(data_set[data_set.drug_name==drug])
    return list_of_drug_dfs

In [113]:
drug_dfs = split_dataframe_by_drug(drug_list, data_set)

In [122]:
def batch_split_xy(list_of_drug_dfs):
    list_x = []
    list_y = []
    for df in list_of_drug_dfs:
        list_x.append(df.iloc[:,0:-4])
        list_y.append(df.iloc[:,-1])
    return list_x, list_y

In [123]:
list_x, list_y = batch_split_xy(drug_dfs)

In [124]:
len(list_x)

3

In [126]:
len(list_y[2]) # correct

46

In [150]:
def batch_train_test_split(list_x, list_y):
    x_trains=[]
    x_tests = []
    y_trains = []
    y_tests = []
    for i, df in enumerate(list_x):
        X_train, X_test, y_train, y_test= train_test_split(df, list_y[i])
        x_trains.append(X_train)
        x_tests.append(X_test)
        y_trains.append(y_train)
        y_tests.append(y_test)
    return x_trains, x_tests, y_trains, y_tests
    
    

In [151]:
x_trains, x_tests, y_trains, y_tests = batch_train_test_split(list_x, list_y)

In [157]:
for train_set in y_trains: 
    print(len(train_set))

46
45
34


In [158]:
for test_set in y_tests: 
    print(len(test_set))

16
15
12
